# Spin selection. Neutrons that do not have the desired spin do not pass through this filter.

In [ ]:
import numpy as np
from qutip import *
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import subprocess
import os

In [ ]:
def folder(name="ss_tmp"):
    """
    Checks if a folder with the given name exists. If it does not, the folder is created.
    Parameters:
    name (str): The name of the folder to check and potentially create.
    """
    if not os.path.exists(name):
        os.makedirs(name)
    
def create_video(path, name, fps=30):
    command = f'ffmpeg -framerate {fps} -i {path}/frame_%d.png -c:v libx264 -pix_fmt yuv420p {name}.mp4'
    subprocess.run(command, shell=True, check=True)

def create_gif(path, name):
    command = f'ffmpeg -framerate 30 -i {path}/frame_%d.png {name}.gif'
    subprocess.run(command, shell=True, check=True)

def display_gif(fn):
    return display.HTML('<img src="{}">'.format(fn))

def display_video(video_path):
    return display.Video(video_path)

In [ ]:
n_vectors = 10                                           # Define the number of vectors
psi_list = [rand_ket_haar(2) for i in range(n_vectors)]  # List of random polorized spins
psi_up = basis(2, 0)                                     # Up state

H = 10*sigmaz()
gamma = 0.333
c_ops = [3*gamma*sigmaz() + 2*gamma*sigmax() + 2*gamma*sigmay()]
times = np.linspace(0, 1, 200)

result = [mesolve(H, psi_list[i], times, c_ops, ) for i in range(n_vectors)]

In [ ]:
folder()

In [ ]:

b = Bloch()
# b.vector_color = ['green']  # Set the color of vector. UPD: Doesn't work properly with the new version

b.zlabel = ['|↑⟩', '|↓⟩']
b.point_marker = ['o']  # Set all point markers to circles
b.point_size = [30]  # Set size of the points

# Animate the vectors
for i in range(len(times)):
    b.clear()  # Clear the Bloch sphere for each frame
    
    # Add decaying vectors
    for res in result:
        current_vector = res.states[i]
        b.add_states(current_vector, 'vector')
        
    
    # Add the nondisappearing vector
    b.add_states(psi_up,'vector')
    
    b.save(f'ss_tmp/frame_{i}.png', format='png')

In [ ]:
create_gif('ss_tmp', 'ss_gif')